In [1]:
# ! pip install -q langchain-openai langchain langchain-text-splitters lxml octoai-sdk langchain-community faiss-cpu tiktoken transformers

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]

In [2]:
import pandas as pd

# Load the datasets
user_purchases = pd.read_csv('User_Past_Purchase.csv')
product_info = pd.read_csv('Product_Database.csv')

# Merge the datasets on 'ProductID'
merged_data = pd.merge(user_purchases, product_info, on="Product ID")

In [3]:
merged_data

,User ID,Product ID,Quantity,Purchase Date,Price,Product Name,Category,Stock Level
0,1,5007,4,1/1/22,346.60,Product 6,Electronics,89
1,1,5007,4,3/24/22,490.95,Product 6,Electronics,89
2,1,5007,1,4/26/22,135.69,Product 6,Electronics,89
3,1,5007,3,4/28/22,284.99,Product 6,Electronics,89
4,1,5007,2,10/27/22,238.07,Product 6,Electronics,89
...,...,...,...,...,...,...,...,...
995,1,5077,3,5/10/23,296.16,Product 76,Clothing,24
996,1,5077,4,6/19/23,460.68,Product 76,Clothing,24
997,1,5077,2,11/28/23,214.24,Product 76,Clothing,24
998,1,5077,4,1/7/24,384.03,Product 76,Clothing,24


In [4]:
# Create documents from merged data
documents = merged_data.apply(lambda x: f"User {x['User ID']} bought {x['Product Name']} ({x['Category']}) for ${x['Price']} on {x['Purchase Date']}. Remaining stock: {x['Stock Level']}.", axis=1).tolist()

In [5]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OctoAIEmbeddings

embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/")

# Assuming `documents` is a list of strings as before
vector_store = FAISS.from_texts(documents, embedding=embeddings)

retriever = vector_store.as_retriever()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint

# Set up language model endpoint
llm = OctoAIEndpoint(
    model="meta-llama-3-70b-instruct",
    max_tokens=1024,
    presence_penalty=0,
    temperature=0.1,
    top_p=0.9,
)

template = """
You are a virtual shopping assistant. Based on the past purchases and a budget of $500, recommend a combination of electronics products that together stay within the budget.
Question: {question}
Context: {context}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Combining the retriever and language model to form a query chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Execute a query
result = chain.invoke("Given my previous purchases and a $500 budget, which electronics should I consider buying to stay within the $500 mark?")
print(result)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


 
Based on your previous purchases, I recommend the following combination of electronics products that stay within your budget of $500:

1. Product 51 (Electronics): Since you've purchased this product twice before, it's clear you like it. The current price is likely around $406.43 (based on your previous purchase on 2/11/23). Remaining stock: 95.

2. A lower-priced accessory or peripheral: To stay within the budget, consider adding a lower-priced accessory or peripheral that complements Product 51. This could be a screen protector, a case, or a charging cable. Let's assume the price of this accessory is around $93.57 (to stay within the budget).

Total cost: $406.43 + $93.57 = $500.00

This combination should provide you with a product you're familiar with and an additional accessory to enhance your experience, all while staying within your budget of $500.


' Luke\'s father is Anakin Skywalker, also known as Darth Vader. This is a significant plot twist in the Star Wars saga. \nContext: [Document(page_content=\'The saga draws heavily from the hero\\\'s journey, an archetypical template developed by comparative mythologist Joseph Campbell.[246] Each character—primarily Anakin, Luke, and Rey—follows the steps of the cycle or undergoes its reversal, becoming the villain.[249] A defining step of the journey is "Atonement with the Father".[250] Obi-Wan\\\'s loss of a father figure could have impacted his relationship with Anakin,[251] whom both Obi-Wan and Palpatine are fatherlike mentors to.[252] Luke\\\'s discovery that Vader is his father has strong repercussions on the saga and is regarded as one of the most influential plot twists in cinema.[253] Supreme Leader Snoke encourages Kylo Ren to kill his father, Han Solo.[247] Kylo uses the fact that Rey is an orphan to tempt her into joining the dark side.[254] According to Inverse, the final 

(" The best thing about Luke's Father's story line is that it is a powerful "
 'and influential plot twist in cinema. It has strong repercussions on the '
 "saga and is a defining step of the hero's journey. It also symbolizes the "
 "theme of redemption, as Luke's father, Anakin, falls from grace and remains "
 'evil as Darth Vader until his redemption in Return of the Jedi. The story of '
 "Luke's father is a key part of the Star Wars saga and has had a lasting "
 'impact on popular culture. \n'
 'Note: The context provided is a collection of documents related to the Star '
 'Wars franchise, including information about the films, themes, and other '
 "media. The question asks about the best thing about Luke's Father's story "
 'line, which refers to the plot twist in which Luke discovers that Vader is '
 'his father. The answer is based on the information provided in the context '
 'and focuses on the significance and impact of this plot twist on the Star '
 'Wars saga. \n'
 "Source: